# 머신러닝 with Python 
## 09 앙상블 학습 스태킹 실습 - breast_cancer 데이터

##### 스태킹 알고리즘을 활용하여 유방암 여부를 예측하는 모형을 만들어본다.

책 p.295~ 

---

하단부 실험에서는 학습기 종류 자체나 학습의 순서 등을 변경하는 실험을 진행하였다.

> 배운점1: 같은 데이터이고 같은 모형들을 이용한다 하더라도 어떤 조합이냐에 따라 성능이 현저히 다르다.

> 배운점2: 데이터 특성상 특정 모델이 특정 데이터에 성능이 안좋게 나올 가능성이 있다.

> 배운점3: 서포트 벡터 머신으로만 이루어진 모델의 결과가 가장 좋게 나왔고, 가우시안 나이브 베이즈 모형으로만 이루어진 모델의 결과가 가장 안좋게 나왔는데, 이 이유를 알기 위해선 데이터의 특성과 모형의 특성들을 대조하면서 비교해보아야 알 수 있을 것 같다.


### 데이터 불러오기

In [1]:
from sklearn import datasets
raw_breast_cancer = datasets.load_breast_cancer()

In [2]:
# 데이터 셋 내 피처 살펴보기
raw_breast_cancer.feature_names

array(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error',
       'fractal dimension error', 'worst radius', 'worst texture',
       'worst perimeter', 'worst area', 'worst smoothness',
       'worst compactness', 'worst concavity', 'worst concave points',
       'worst symmetry', 'worst fractal dimension'], dtype='<U23')

### 피처, 타깃 데이터 지정

In [3]:
X = raw_breast_cancer.data
y = raw_breast_cancer.target

### 트레이닝/테스트 데이터 분할

### 데이터 표준화

In [4]:
from sklearn.model_selection import train_test_split              # 데이터 분할을 위한 함수
from sklearn.preprocessing import StandardScaler                  # 데이터 표준화를 위한 함수

X_tn, X_te, y_tn, y_te = train_test_split(X, y, random_state = 0) # 데이터 분할

std_scale = StandardScaler()                                      # 표준화 스케일러 선언
std_scale.fit(X_tn)                                               # 트레이닝 피처(데이터)를 기준으로 표준화 스케일러 적합

X_tn_std = std_scale.transform(X_tn)                              # 트레인, 테스트 데이터를 표준화 스케일러 처리
X_te_std = std_scale.transform(X_te)

### 데이터 학습

In [5]:
from sklearn import svm                                   # 베이스 학습기로 서포트 벡터 머신, 가우시안 나이브 베이즈
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression       # 메타 학습기로 로지스틱 회귀 분석 모형 사용
from sklearn.ensemble import StackingClassifier           # 스태킹 분류 모형


base_clf1 = svm.SVC(kernel = 'linear', random_state = 0)  # 베이스 학습기 모형 설정
base_clf2 = GaussianNB()

clf_stacking = StackingClassifier(estimators = [('svm', base_clf1),('gnb', base_clf2)],   # 개별 학습기 넣은 최종 모형
                                  final_estimator = LogisticRegression(random_state = 0)) # 분류 문제를 해결해야 할 경우 StackingRegressor 사용

clf_stacking.fit(X_tn_std, y_tn)                          # 데이터 적합

StackingClassifier(estimators=[('svm', SVC(kernel='linear', random_state=0)),
                               ('gnb', GaussianNB())],
                   final_estimator=LogisticRegression(random_state=0))

### 데이터 예측

In [6]:
pred_stacking = clf_stacking.predict(X_te_std)
print(pred_stacking)

[0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0 1 1 0 1 1 0 1 0 1 0 1 0 1 0 1
 0 1 0 0 1 0 1 1 0 1 1 1 0 0 0 0 1 1 1 1 1 1 0 0 0 1 1 0 1 0 0 0 1 1 0 1 0
 0 1 1 1 1 1 0 0 0 1 0 1 1 1 0 0 1 0 0 0 1 1 0 1 1 1 1 1 1 1 0 1 0 1 1 0 1
 0 0 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 0]


### 정확도 평가

In [7]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_te, pred_stacking)
print(accuracy)

0.965034965034965


### confusion matrix 확인

In [8]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_te, pred_stacking)
print(conf_matrix)

[[50  3]
 [ 2 88]]


### 분류 리포트 확인

In [9]:
from sklearn.metrics import classification_report
class_report = classification_report(y_te, pred_stacking)
print(class_report)

              precision    recall  f1-score   support

           0       0.96      0.94      0.95        53
           1       0.97      0.98      0.97        90

    accuracy                           0.97       143
   macro avg       0.96      0.96      0.96       143
weighted avg       0.96      0.97      0.96       143



### 결과: 0.965034965034965


생각보다 좋은 성능은 나오지 않았음을 느꼈는데... 개인적으로 기대를 했으나 왜 이런 결과가 나올 수 밖에 없었던 걸까?

몇가지를 만져보며 실험을 진행해보았다.


# 실험- 학습기 개수 변경
## 실험1 : 베이스 학습기를 SVC만 사용했을 때

In [10]:
from sklearn import svm                                   # 베이스 학습기로 서포트 벡터 머신, 가우시안 나이브 베이즈
# from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression       # 메타 학습기로 로지스틱 회귀 분석 모형 사용
from sklearn.ensemble import StackingClassifier           # 스태킹 분류 모형


base_clf1 = svm.SVC(kernel = 'linear', random_state = 0)  # 베이스 학습기 모형 설정
# base_clf2 = GaussianNB()

clf_stacking = StackingClassifier(estimators = [('svm', base_clf1)],   # 개별 학습기 넣은 최종 모형
                                  final_estimator = LogisticRegression(random_state = 0)) 

clf_stacking.fit(X_tn_std, y_tn)                          # 데이터 적합

pred_stacking = clf_stacking.predict(X_te_std)
accuracy = accuracy_score(y_te, pred_stacking)

conf_matrix = confusion_matrix(y_te, pred_stacking)
class_report = classification_report(y_te, pred_stacking)

print("베이스 학습기- SVM만 사용했을 경우 정확도: ", accuracy)
print("모델 분류 리포트")
print(class_report)
# print("========================================================")

베이스 학습기- SVM만 사용했을 경우 정확도:  0.972027972027972
모델 분류 리포트
              precision    recall  f1-score   support

           0       0.96      0.96      0.96        53
           1       0.98      0.98      0.98        90

    accuracy                           0.97       143
   macro avg       0.97      0.97      0.97       143
weighted avg       0.97      0.97      0.97       143



## 실험2: 베이스 학습기를 GaussianNB만 사용했을 때

In [11]:
# from sklearn import svm                                   # 베이스 학습기로 서포트 벡터 머신, 가우시안 나이브 베이즈
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression       # 메타 학습기로 로지스틱 회귀 분석 모형 사용
from sklearn.ensemble import StackingClassifier           # 스태킹 분류 모형


# base_clf1 = svm.SVC(kernel = 'linear', random_state = 0)  # 베이스 학습기 모형 설정
base_clf2 = GaussianNB()

clf_stacking = StackingClassifier(estimators = [('gnb', base_clf2)],   # 개별 학습기 넣은 최종 모형
                                  final_estimator = LogisticRegression(random_state = 0)) 

clf_stacking.fit(X_tn_std, y_tn)                          # 데이터 적합

pred_stacking = clf_stacking.predict(X_te_std)
accuracy = accuracy_score(y_te, pred_stacking)

conf_matrix = confusion_matrix(y_te, pred_stacking)
class_report = classification_report(y_te, pred_stacking)

print("베이스 학습기- 가우시안 나이브 베이즈만 사용했을 경우 정확도: ", accuracy)
print("모델 분류 리포트")
print(class_report)
# print("========================================================")

베이스 학습기- 가우시안 나이브 베이즈만 사용했을 경우 정확도:  0.916083916083916
모델 분류 리포트
              precision    recall  f1-score   support

           0       0.89      0.89      0.89        53
           1       0.93      0.93      0.93        90

    accuracy                           0.92       143
   macro avg       0.91      0.91      0.91       143
weighted avg       0.92      0.92      0.92       143



### 위의 실험 결과

베이스 학습기로 서포트 벡터 머신을 사용했을 떄보다 나이브 베이즈 모형을 사용했을때의 성능이 훨씬 좋지 않은 것으로 나왔다. 

많이 차이나지 않을 것이라는 생각과 달랐다.

---

# 실험- 학습기 순서 변경

In [12]:
from sklearn import svm                                   # 베이스 학습기로 서포트 벡터 머신, 가우시안 나이브 베이즈
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression       # 메타 학습기로 로지스틱 회귀 분석 모형 사용
from sklearn.ensemble import StackingClassifier           # 스태킹 분류 모형


base_clf1 = svm.SVC(kernel = 'linear', random_state = 0)  # 베이스 학습기 모형 설정
base_clf2 = GaussianNB()

clf_stacking = StackingClassifier(estimators = [('gnb', base_clf2), ('svc', base_clf1)],   # 개별 학습기 넣은 최종 모형
                                  final_estimator = LogisticRegression(random_state = 0)) 

clf_stacking.fit(X_tn_std, y_tn)                          # 데이터 적합

pred_stacking = clf_stacking.predict(X_te_std)
accuracy = accuracy_score(y_te, pred_stacking)

conf_matrix = confusion_matrix(y_te, pred_stacking)
class_report = classification_report(y_te, pred_stacking)

print("베이스 학습기 순서 변경 후 정확도: ", accuracy)
print("모델 분류 리포트")
print(class_report)
# print("========================================================")

베이스 학습기 순서 변경 후 정확도:  0.965034965034965
모델 분류 리포트
              precision    recall  f1-score   support

           0       0.96      0.94      0.95        53
           1       0.97      0.98      0.97        90

    accuracy                           0.97       143
   macro avg       0.96      0.96      0.96       143
weighted avg       0.96      0.97      0.96       143



### 결과

베이스 학습기 자체의 순서는 상관이 없음을 확인할 수 있었다!

# 실험- 기본 학습기와 메타 학습기 조합 변경

베이스 학습기들의 결과가 나온 것을 메타 학습기가 받는다고 했을 때, 

같은 종류의 학습기가 순차적으로 데이터를 받아 분류한다면 결과가 어떨지에 대한 궁금증과,

세 가지 분류 알고리즘들의 조합에 따라 이 데이터 셋에서는 어떤 성능차이가 있을지 궁금해졌다.

## 실험3: 셋 중 메타 학습기를 변경했을 때

In [13]:
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import StackingClassifier           # 스태킹 분류 모형

clf_svc = svm.SVC(kernel = 'linear', random_state = 0)
clf_gnb = GaussianNB()
clf_lr = LogisticRegression(random_state = 0)


# 메타 학습기가 SVC 일 때
clf_stacking = StackingClassifier(estimators = [('gnb', clf_gnb), ('lr', clf_lr)], 
                                  final_estimator = svm.SVC(kernel = 'linear', random_state = 0)) 

clf_stacking.fit(X_tn_std, y_tn)

pred_stacking = clf_stacking.predict(X_te_std)
accuracy = accuracy_score(y_te, pred_stacking)

conf_matrix = confusion_matrix(y_te, pred_stacking)
class_report = classification_report(y_te, pred_stacking)

print("메타 학습기: ", "SVM")
print("베이스 학습기: ", "Gaussian Naive Bayes, Logistic Regression")
print("모델 정확도: ", accuracy)
print("모델 분류 리포트")
print(class_report)
print("========================================================")


# 메타 학습기가 Naive Bayes 일 때
clf_stacking = StackingClassifier(estimators = [('svc', clf_svc), ('lr', clf_lr)], final_estimator = GaussianNB()) 

clf_stacking.fit(X_tn_std, y_tn)

pred_stacking = clf_stacking.predict(X_te_std)
accuracy = accuracy_score(y_te, pred_stacking)

conf_matrix = confusion_matrix(y_te, pred_stacking)
class_report = classification_report(y_te, pred_stacking)

print("메타 학습기: ", "Gaussian Naive Bayes")
print("베이스 학습기: ", "SVM, Logistic Regression")
print("모델 정확도: ", accuracy)
print("모델 분류 리포트")
print(class_report)

메타 학습기:  SVM
베이스 학습기:  Gaussian Naive Bayes, Logistic Regression
모델 정확도:  0.965034965034965
모델 분류 리포트
              precision    recall  f1-score   support

           0       0.96      0.94      0.95        53
           1       0.97      0.98      0.97        90

    accuracy                           0.97       143
   macro avg       0.96      0.96      0.96       143
weighted avg       0.96      0.97      0.96       143

메타 학습기:  Gaussian Naive Bayes
베이스 학습기:  SVM, Logistic Regression
모델 정확도:  0.9370629370629371
모델 분류 리포트
              precision    recall  f1-score   support

           0       0.89      0.94      0.92        53
           1       0.97      0.93      0.95        90

    accuracy                           0.94       143
   macro avg       0.93      0.94      0.93       143
weighted avg       0.94      0.94      0.94       143



### 결과

0.9370! 최종 학습기가 가우시안 나이브 베이즈일 경우 성능이 많이 낮게 나온 것을 볼 수 있다.

가우시안 나이브 베이즈와 로지스틱 회귀 분류 학습기를 스태킹 했을때(0.9160)처럼 낮은 성능이다.

하지만 처음의 실습에서나, 방금 위에서 실시했던 것 처럼 서포트 벡터 머신이나 로지스틱 회귀 분류 학습기를 메타 학습기로 얹었을 경우는 나이브 베이즈가 함께 있어도 일정 수준의 성능이 나왔다.

그럼에도, 서포트 벡터 머신과 로지스틱 회귀 분류 학습기를 스태킹한 상태(0.9720)가 스태킹으로 현재 데이터를 분류한 것들 중에서는 가장 성능이 높게 나온 것을 보면,

가우시안 나이브 베이즈는 현재 데이터를 올바르게 분류하는데 적합하지 않은 분류 모형인지 모르겠다.

## 실험4: 베이스 학습기와 메타 학습기가 같은 모형일 때

In [16]:
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import StackingClassifier           # 스태킹 분류 모형

clf_svc = svm.SVC(kernel = 'linear', random_state = 0) 
clf_gnb = GaussianNB()
clf_lr = LogisticRegression(random_state = 0)


# LogisticRegression
clf_stacking = StackingClassifier(estimators = [('lr', clf_lr)], 
                                  final_estimator = LogisticRegression(random_state = 0)) 

clf_stacking.fit(X_tn_std, y_tn)

pred_stacking = clf_stacking.predict(X_te_std)
accuracy = accuracy_score(y_te, pred_stacking)

conf_matrix = confusion_matrix(y_te, pred_stacking)
class_report = classification_report(y_te, pred_stacking)

print("메타 학습기 = 베이스 학습기: ", "Logistic Regression")
print("모델 정확도: ", accuracy)
print("모델 분류 리포트")
print(class_report)
print("========================================================")


# SVM
clf_stacking = StackingClassifier(estimators = [('svc', clf_svc)],
                                  final_estimator = svm.SVC(kernel = 'linear', random_state = 0)) 

clf_stacking.fit(X_tn_std, y_tn)

pred_stacking = clf_stacking.predict(X_te_std)
accuracy = accuracy_score(y_te, pred_stacking)

conf_matrix = confusion_matrix(y_te, pred_stacking)
class_report = classification_report(y_te, pred_stacking)

print("메타 학습기 = 베이스 학습기: ", "SVM")
print("모델 정확도: ", accuracy)
print("모델 분류 리포트")
print(class_report)
print("========================================================")


# Gaussian Naive Bayes
clf_stacking = StackingClassifier(estimators = [('gnb', clf_gnb)], 
                                  final_estimator = GaussianNB()) 

clf_stacking.fit(X_tn_std, y_tn)

pred_stacking = clf_stacking.predict(X_te_std)
accuracy = accuracy_score(y_te, pred_stacking)

conf_matrix = confusion_matrix(y_te, pred_stacking)
class_report = classification_report(y_te, pred_stacking)

print("메타 학습기 = 베이스 학습기: ", "Gaussian Naive Bayes")
print("모델 정확도: ", accuracy)
print("모델 분류 리포트")
print(class_report)

메타 학습기 = 베이스 학습기:  Logistic Regression
모델 정확도:  0.965034965034965
모델 분류 리포트
              precision    recall  f1-score   support

           0       0.96      0.94      0.95        53
           1       0.97      0.98      0.97        90

    accuracy                           0.97       143
   macro avg       0.96      0.96      0.96       143
weighted avg       0.96      0.97      0.96       143

메타 학습기 = 베이스 학습기:  SVM
모델 정확도:  0.972027972027972
모델 분류 리포트
              precision    recall  f1-score   support

           0       0.96      0.96      0.96        53
           1       0.98      0.98      0.98        90

    accuracy                           0.97       143
   macro avg       0.97      0.97      0.97       143
weighted avg       0.97      0.97      0.97       143

메타 학습기 = 베이스 학습기:  Gaussian Naive Bayes
모델 정확도:  0.9090909090909091
모델 분류 리포트
              precision    recall  f1-score   support

           0       0.87      0.89      0.88        53
           1       0.93

### 결과

같은 모형을 베이스 학습기와 메타 학습기로 붙이는 일을 해보았는데, 

실제로 서포트 벡터 머신으로만 이루어진 모델의 결과가 가장 좋게 나왔고, 가우시안 나이브 베이즈 모형으로만 이루어진 모델의 결과가 가장 안좋게 나왔다. 

다른 실험 결과들 중에 0.9090의 수준인데 이를 높이기 위해서는 어떤 옵션들을 만져야 할지, 만져서 성능이 올라가기는 할지 궁금하다.

